# Implementing VGG-16 in Python using PyTorch

## Importing libraries

In [6]:
import torch
import torchvision
import torchvision.transforms as transforms

my_transform = transforms.Compose(
                      [transforms.Resize((224,224)),
                      transforms.ToTensor(),
                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

## Defining model

In [7]:
import torch.nn as nn

in_size = 3
out_size = 24

In [8]:
class VGG_16(nn.Module):
    def __init__(self):
        super(VGG_16, self).__init__()

        #224x224x3 image
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_size, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        #112x112x128 image
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        #56x56x256 image
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        #28x28x512 image
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        #14x14x512 image
        self.conv5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
    
        #7x7x512 image
        self.fc1 = nn.Linear(7*7*512, 4096)
        self.r1 = nn.ReLU()
        self.d1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(4096, 4096)
        self.r2 = nn.ReLU()
        self.d2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(4096, out_size)
        self.sm = nn.Softmax(dim=1)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        out = self.r1(out)
        out = self.d1(out)
        out = self.fc2(out)
        out = self.r2(out)
        out = self.d2(out)
        out = self.fc3(out)
#         out = self.sm(out)
        return out

## Loading dataset (fruits-360, 24 classes)

In [9]:
trainset = torchvision.datasets.ImageFolder(root='/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Training/', transform=my_transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                          num_workers=2,
                                          batch_size=32,
                                          shuffle=True)
testset = torchvision.datasets.ImageFolder(root='/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Test/', transform=my_transform)
testloader = torch.utils.data.DataLoader(testset,
                                         num_workers=2,
                                         batch_size=32,
                                         shuffle=False)

model = VGG_16().to('cuda')

learning_rate = 0.001

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

## Training the model

In [11]:
total_step = len(trainloader)
start_epoch = 0
num_epochs_total = 5

for epoch in range(start_epoch, num_epochs_total):
    for i, (images, labels) in enumerate(trainloader):
        
        outputs = model(images.to('cuda'))
        loss = criterion(outputs.to('cuda'), labels.to('cuda'))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs_total, i+1, total_step, loss.item()))

Epoch [1/5], Step [20/195], Loss: 0.0244
Epoch [1/5], Step [40/195], Loss: 0.0253
Epoch [1/5], Step [60/195], Loss: 0.0165
Epoch [1/5], Step [80/195], Loss: 0.0410
Epoch [1/5], Step [100/195], Loss: 0.0024
Epoch [1/5], Step [120/195], Loss: 0.0088
Epoch [1/5], Step [140/195], Loss: 0.0069
Epoch [1/5], Step [160/195], Loss: 0.0380
Epoch [1/5], Step [180/195], Loss: 0.0170
Epoch [2/5], Step [20/195], Loss: 0.0040
Epoch [2/5], Step [40/195], Loss: 0.0105
Epoch [2/5], Step [60/195], Loss: 0.0083
Epoch [2/5], Step [80/195], Loss: 0.1958
Epoch [2/5], Step [100/195], Loss: 0.0111
Epoch [2/5], Step [120/195], Loss: 0.0060
Epoch [2/5], Step [140/195], Loss: 0.0088
Epoch [2/5], Step [160/195], Loss: 0.0091
Epoch [2/5], Step [180/195], Loss: 0.0051
Epoch [3/5], Step [20/195], Loss: 0.0024
Epoch [3/5], Step [40/195], Loss: 0.0022
Epoch [3/5], Step [60/195], Loss: 0.0064
Epoch [3/5], Step [80/195], Loss: 0.0038
Epoch [3/5], Step [100/195], Loss: 0.0070
Epoch [3/5], Step [120/195], Loss: 0.0024
Epoc

## Finding the accuracy of trained model

In [12]:
model.eval()  
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:

        outputs = model(images.to("cuda"))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.to("cuda") == labels.to("cuda")).sum().item()

    print('Accuracy of the model on the test images: {}%'\
          .format(100 * correct / total))

Accuracy of the model on the test images: 100.0%
